<a href="https://colab.research.google.com/github/tobytoyin/sec-10q-msc-report/blob/main/1b_Extract_Item_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install textunits==0.0.2
!pip install html5lib
import pandas as pd 
import os 
from bs4 import BeautifulSoup
import re
from textunits.textunits import *


dl_dir = '/content/drive/MyDrive/Aston/Term 3/data'
data_dir = f'{dl_dir}/10-Q-original'
save_dir = f'{dl_dir}/10-Q-item-2'
file_name = 'edgar-filing-new.csv'
# read downloaded records 
edgar_filings = pd.read_csv(f'{dl_dir}/{file_name}', 
                            index_col=['acc_num'], 
                            dtype=object)

print(edgar_filings.shape)


# selected_company = pd.read_csv('/content/drive/MyDrive/Aston/Term 3/data/selected_companies.csv')
# selected_company['GICS Sector'].unique()

(16672, 21)


In [ ]:
# edgar_filings = edgar_filings.reset_index().merge(selected_company[['Symbol', 'GICS Sector']], left_index=True, left_on='sym', right_on='Symbol').set_index('acc_num')
edgar_filings = edgar_filings[edgar_filings['GICS Sector'].isin([
  'Health Care', 'Information Technology',
  'Communication Services', 'Consumer Discretionary', 'Utilities', 'Real Estate', 'Consumer Staples'
])]
edgar_filings.tail()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,cik,downloaded,filing_date,filing_dir,filing_url,report_date,section_1a_length,section_1a_path,section_2_length,section_2_path,sym,type,section_2_headline,section_2_endline,filed_as_of_date,Symbol,GICS Sector
acc_num,,,,,,,,,,,,,,,,,,,,,
0001555280-14-000274,22945,22945.0,0.0,NaN,1555280.0,ZTS-0001555280-14-000274.txt,2014-08-12,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...,NaN,0.0,NaN,0.0,NaN,ZTS,10-Q,NaN,NaN,NaN,ZTS,Health Care
0001555280-14-000213,22946,22946.0,0.0,NaN,1555280.0,ZTS-0001555280-14-000213.txt,2014-05-13,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...,NaN,0.0,NaN,0.0,NaN,ZTS,10-Q,NaN,NaN,NaN,ZTS,Health Care
0001555280-13-000114,22947,22947.0,0.0,NaN,1555280.0,ZTS-0001555280-13-000114.txt,2013-11-13,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...,NaN,0.0,NaN,0.0,NaN,ZTS,10-Q,NaN,NaN,NaN,ZTS,Health Care
0001555280-13-000042,22948,22948.0,0.0,NaN,1555280.0,ZTS-0001555280-13-000042.txt,2013-08-14,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...,NaN,0.0,NaN,0.0,NaN,ZTS,10-Q,NaN,NaN,NaN,ZTS,Health Care
0001555280-13-000020,22949,22949.0,0.0,NaN,1555280.0,ZTS-0001555280-13-000020.txt,2013-05-15,https://www.sec.gov/Archives/edgar/data/155528...,https://www.sec.gov/Archives/edgar/data/155528...,NaN,0.0,NaN,0.0,NaN,ZTS,10-Q,NaN,NaN,NaN,ZTS,Health Care


In [ ]:
def decompose_table(soup, decompose_tag, exclude):
  """Function to remove tables from the soup
  
  Params: 
    soup - BeautifulSoup object 
    exclude - re.compile(...) object 

  Return:
    soup - after decomposing table
  """
  soup = soup
  for tag in soup(decompose_tag): 
    # check if the table contains any cell having long paragraphs
    if any(list(map(lambda x: len(x.text.split()) > 200, tag('td')))):
      continue

    if exclude and not exclude.search(tag.text.lower().strip()):
      tag.decompose()

  return soup

In [ ]:
## these regex determine the head of the section 
# head_item_regex = re.compile('^\s*(item)\s?2')
head_item_regex = re.compile('^\s*management.s\s*discussion\s')
head_title_regex = re.compile('^\s*(item\s*\d?\.?)?\s*management.s\s*discussion\s*and\s*analysis')

## these regex determine the end of the section
end_item_regex = re.compile('^\s*(item)\s*[34]')
end_title_regex = re.compile('^\s*(item\s*\d?\.?)?\s*quantitative')

## avoid decompose table item
header_regex = re.compile('^\s*([iI][tT][eE][mM]|management.s discussion and analysis)')

def end_nodes_custom(tag): return end_node_with_text(
    tag, end_nodes=['div', 'p', 'td', 'font', 'h1', 'h2', 'h3', 'h4', 'h5'])

def create_soup(path):
    with open(path, 'r', encoding='utf-8') as f:
      text = f.read()

    try: 
      soup = BeautifulSoup(text, 'html.parser')
      # soup = BeautifulSoup(text, 'html5lib')
      soup = decompose_table(soup, 'table', exclude=header_regex)
    except Exception as e:
      print(e)
      return None
    return soup 

def get_meta(path):
    """Retrieve the meta from the Header"""
    header_text = ''

    meta_lines = []
    print('Read txt File')
    ## file is read from html, 
    # need to re-read the txt file 
    with open(path, 'r') as f:
      print('Loading Meta.', end='.')
      for i in range(1, 100):
        line = f.readline()
        meta_lines.append(line)
        if '</SEC-HEADER>' in line:
          print('')
          break

    # use lines to create header_text
    header_text = ' '.join(meta_lines)
    
    metas = {}
    # locate the meta
    groups = re.findall('(.*:.*\s{0,5})', header_text)
    for group in groups: 
      try: 
        key, value = group.split(':', maxsplit=1)
      except: 
        key = value = ''

      metas = {**metas, 
          key.lower().strip(): value.lower().strip()
      }
    return metas

# def create_contents(soup):
#   contents = list(filter(None, soup.get_text().strip().split('\n')))

#   # print(contents)
#   return contents


def create_contents(soup):
  try: 
    textunits = TextUnits(end_nodes_custom)
    if not soup:
      return []

    textunits.add_soup(soup)

    contents = []

    for unit in textunits.units: 
      line = unit.text.strip()
      if line != '': 
        contents.append(line)

    return contents

  except Exception as e:
    print(e)
    return []

def find_section_2(contents: list): 
  head_cut_point = 0
  end_cut_point = 0
  head_line = ''
  end_line = ''


  for id, line in enumerate(contents): 
    text = line.lower().strip()
    next_10_lines = ' '.join(contents[id: id+5])
    ## locate the head of Item 2 for once 
    if head_cut_point == 0 and (head_item_regex.search(text) or head_title_regex.search(text)):
      # check the word counts of the next 10 lines
      # next10_word_counts = sum(map(lambda x: len(x.split()), contents[id: id+10]))
      next10_word_counts = sum(map(lambda x: len(x.split()), next_10_lines))
      
      # if next 10 lines have more than 200 words 
      if next10_word_counts >= 120:
        print("starting point: ", line)
        head_cut_point = id
        head_line = (line + ' ' + next_10_lines)[0:500]
      else: 
        print(contents[id: id+10])
        print('rejected : ', str(next10_word_counts)[0::])

    ## locate the head of Item 3
    if head_cut_point > 0 and (
        end_item_regex.search(text) or end_title_regex.search(text)
      ):
      print("cutting point: ", line)
      end_cut_point = id
      end_line = (line + ' ' + next_10_lines)[0:500]
      break

  if head_cut_point == 0 or end_cut_point == 0:
    
    return ' ', (head_line, end_line)
  print('----', head_line)
  print('----', end_line)
  return contents[head_cut_point: end_cut_point], (head_line, end_line)

##### filter lines #####
filter_regex = [
  '^\s*table of contents\s*$', '[\-_\.]{3, }', '^\d{1,3}\s{0,10}', '^\s*item.{0,10}\s{0,10}'
]

def filter_line(line, filter_regex):
  line = re.sub(r'[^\x00-\x7F]+', ' ', line).strip()

  pattern = '(' + '|'.join(filter_regex) + ')'
  if re.search(pattern, line.lower()):
    return '' 
  return line

In [ ]:
def create_doc(path): 
  ## run the records
  soup = create_soup(path)
  meta = get_meta(path)

  # generate list of contents lines 
  contents = create_contents(soup)
  section2, head_end_lines = find_section_2(contents)

  # filter out table of contents and page numbers 
  refine_section2 = []
  for line in section2:
    refine_section2.append(filter_line(line, filter_regex))

  # remove empty 
  doc = ' '.join(list(filter(None, refine_section2)))
  return doc, meta, head_end_lines

In [ ]:
test = False

## build a for loop to look up all the sec filings ## 
i = 0
try: 
  for acc_num, record in edgar_filings.iterrows(): 
    # check if completed 
    if float(record['section_2_length']) > 500:
      continue

    i += 1
    print('---------', acc_num, record["sym"], '---------\n\n')
    file_path = f'{data_dir}/{record["sym"]}/{record["downloaded"]}'

    try: 
      head_end_lines = ('','')
      doc, meta, head_end_lines = create_doc(file_path)
      # report_date = meta['conformed period of report']
      # filed_date = meta['filed as of date']
      report_date = None
      filed_date = None

    except Exception as e: 
      print(e)
      doc = ''
      report_date = None
      filed_date = None

    
    if not test:
      # save to file 
      save_path = f'{save_dir}/{record["sym"]}-{acc_num}.txt'
      with open(save_path, 'w') as f:
        f.write(doc)

      # update status 
    
      edgar_filings.at[acc_num, 'section_2_path'] = save_path
      edgar_filings.at[acc_num, 'section_2_length'] = len(doc.split())
      edgar_filings.at[acc_num, 'section_2_headline'] = head_end_lines[0]
      edgar_filings.at[acc_num, 'section_2_endline'] = head_end_lines[1]
      edgar_filings.at[acc_num, 'report_date'] = report_date
      edgar_filings.at[acc_num, 'filed_as_of_date'] = filed_date

      if i % 30 == 0:
        edgar_filings.to_csv(f'{dl_dir}/{file_name}')

finally:
  pass
  # if not test: 
  #   edgar_filings.to_csv(f'{dl_dir}/{file_name}')

--------- 0000912057-01-005532 ATVI ---------


Read txt File
Loading Meta..
--------- 0000912057-00-049633 ATVI ---------


Read txt File
Loading Meta..
--------- 0000912057-00-036795 ATVI ---------


Read txt File
Loading Meta..
--------- 0000912057-00-006753 ATVI ---------


Read txt File
Loading Meta..
--------- 0001012870-01-502805 AMD ---------


Read txt File
Loading Meta..
--------- 0001012870-01-501532 AMD ---------


Read txt File
Loading Meta..
--------- 0001012870-01-500838 AMD ---------


Read txt File
Loading Meta..
--------- 0001012870-00-005706 AMD ---------


Read txt File
Loading Meta..
--------- 0001012870-00-004459 AMD ---------


Read txt File
Loading Meta..
--------- 0001012870-00-002943 AMD ---------


Read txt File
Loading Meta..
--------- 0000950123-02-010980 AKAM ---------


Read txt File
Loading Meta..
starting point:  Item 2. Management’s Discussion and Analysis of Financial Condition and Results of Operations
cutting point:  Item 3. Quantitative and Qualita

/usr/local/lib/python3.7/dist-packages/bs4/builder/_htmlparser.py:78: UserWarning: unknown status keyword 'IBS' in marked section
  warnings.warn(msg)


local variable 'match' referenced before assignment
Read txt File
Loading Meta..
--------- 0000950132-02-000241 ANSS ---------


Read txt File
Loading Meta..
--------- 0000950132-02-000166 ANSS ---------


Read txt File
Loading Meta..
--------- 0001021408-02-006256 ANSS ---------


Read txt File
Loading Meta..
--------- 0000927016-01-503585 ANSS ---------


Read txt File
Loading Meta..
--------- 0001021408-01-504377 ANSS ---------


Read txt File
Loading Meta..
--------- 0000950132-01-500239 ANSS ---------


Read txt File
Loading Meta..
--------- 0001013462-00-000018 ANSS ---------


Read txt File
Loading Meta..
--------- 0001013462-00-000012 ANSS ---------


Read txt File
Loading Meta..
--------- 0000950132-00-000337 ANSS ---------


Read txt File
Loading Meta..
--------- 0000912057-01-004642 AAPL ---------


Read txt File
Loading Meta..
--------- 0000912057-00-033901 AAPL ---------


Read txt File
Loading Meta..
--------- 0000912057-00-023442 AAPL ---------


Read txt File
Loading Me

/usr/local/lib/python3.7/dist-packages/bs4/builder/_htmlparser.py:78: UserWarning: unknown status keyword 'A' in marked section
  warnings.warn(msg)


local variable 'match' referenced before assignment
Read txt File
Loading Meta..
--------- 0000008670-05-000244 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-05-000118 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-05-000098 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-04-000192 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-04-000085 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-04-000053 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-03-000347 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-03-000082 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-03-000051 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-02-000019 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-02-000005 ADP ---------


Read txt File
Loading Meta..
--------- 0000008670-02-000002 ADP ---------


Read txt File
Loading Meta..
-------

...

cannot unpack non-iterable NoneType object
Read txt File
Loading Meta..
--------- 0000766704-14-000012 WELL ---------


Read txt File
Loading Meta..
starting point:  Management’s Discussion and Analysis of Financial

Condition and Results of Operations
cutting point:  Item 3.
---- Management’s Discussion and Analysis of Financial

Condition and Results of Operations Management’s Discussion and Analysis of Financial

Condition and Results of Operations 25 Item 2. Management’s Discussion and Analysis of Financial

Condition and Results of Operations The
following discussion and analysis is based primarily on the unaudited consolidated
financial statements of Health Care REIT, Inc. for the periods presented and
should be read together with the notes thereto contained in this
---- Item 3. Item 3. Quantitative and Qualitative Disclosures about Market
Risk We are exposed to
various market risks, including the potential loss arising from adverse changes
in interest rates and foreign currency 

/usr/local/lib/python3.7/dist-packages/bs4/builder/_htmlparser.py:78: UserWarning: expected name token at '<![)0A="]*W`^-!\nML+/'
  warnings.warn(msg)


cannot unpack non-iterable NoneType object
Read txt File
Loading Meta..
--------- 0000072903-14-000085 XEL ---------


Read txt File
Loading Meta..
starting point:  Management’s Discussion and Analysis, in Xcel Energy Inc.’s Annual Report on Form 10-K for the year ended
cutting point:  Item 3
---- Management’s Discussion and Analysis, in Xcel Energy Inc.’s Annual Report on Form 10-K for the year ended Management’s Discussion and Analysis, in Xcel Energy Inc.’s Annual Report on Form 10-K for the year ended Dec. 31, 2013 , includes a discussion of accounting policies and estimates that are most significant to the portrayal of Xcel Energy’s financial condition and results, and that require management’s most difficult, subjective or complex judgments.  Each of these has a higher likelihood of
---- Item 3 Item 3 — QUANTITATIVE AND QUALITATIVE DISCLOSURES ABOUT MARKET RISK See Management’s Discussion and Analysis
--------- 0000072903-14-000061 XEL ---------


Read txt File
Loading Meta..
sta

/usr/local/lib/python3.7/dist-packages/bs4/builder/_htmlparser.py:78: UserWarning: expected name token at '<![[Y#O71R>N()_M>;<"'
  warnings.warn(msg)


cannot unpack non-iterable NoneType object
Read txt File
Loading Meta..
--------- 0001193125-15-177116 ZBH ---------


Read txt File
Loading Meta..
starting point:  Management’s Discussion and Analysis of Financial Condition and Results of Operations
cutting point:  Item 3.
---- Management’s Discussion and Analysis of Financial Condition and Results of Operations Management’s Discussion and Analysis of Financial Condition and Results of Operations The following discussion and analysis should be read in conjunction with the condensed consolidated financial statements and corresponding
notes included elsewhere in this Form 10-Q. Certain percentages presented in this discussion and analysis are calculated from the underlying whole-dollar amounts and, therefore, may not rec
---- Item 3. Item 3. Quantitative and Qualitative Disclosures About Market Risk There have been no material
changes from the information provided in our Annual Report on Form 10-K for the year ended December 31, 2014. I

In [ ]:
edgar_filings.to_csv(f'{dl_dir}/{file_name}')